In [1]:
import numpy

In [ ]:
def compareCard(subArray, COLOR):
    sum = 0
    for column in range(subArray.shape()[0]):
        for row in range(subArray.shape()[1]):
            if subArray[column,row] == COLOR:
                sum +=1
    result = sum/ ( subArray.shape()[0]*subArray.shape()[1] )
    return result

In [ ]:
def detection(image, background_color, backside_color):

    BACKGROUND_COLOR = background_color
    BACKSIDE_COLOR = backside_color

    card_shape = (32,32)
    
    card_list = []

    for column in range(image.shape()[0] - card_shape[0]):
        for row in range(image.shape()[1] - card_shape[1]):
            subArray = image[column:column+card_shape[0], row:row+card_shape[1]]
            if compareCard(subArray,BACKGROUND_COLOR) == 0:
                card_list.append(subArray)
    
    for card in card_list:
        if compareCard(card,BACKSIDE_COLOR) == 0:
            return card